### Imports

In [18]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd

### URLs oficiais dos programas sociais

In [19]:
urls = {
    "Bolsa Família": "https://www.gov.br/mds/pt-br/acoes-e-programas/bolsa-familia",
    "BPC": "https://www.gov.br/mds/pt-br/acoes-e-programas/suas/beneficios-assistenciais/beneficio-assistencial-ao-idoso-e-a-pessoa-com-deficiencia-bpc",
    "Pe-de-Meia": "https://www.gov.br/secom/pt-br/acesso-a-informacao/comunicabr/lista-de-acoes-e-programas/pe-de-meia",
}

### Função para extrair texto de uma página

In [20]:
def extrair_texto(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        return soup.get_text(separator=" ").lower()
    else:
        print(f"Erro ao acessar {url}: {response.status_code}")
        return ""

### Função para buscar critérios no texto

In [21]:
def extrair_criterios(programa, texto):
    criterios = {}

    if programa == "Bolsa Família":
        renda = re.search(r"r\$\s*([0-9]+)\s*,?\.?([0-9]*)", texto)
        if renda:
            criterios["renda_per_capita_max"] = float(renda.group(1))
        criterios["cad_unico_required"] = "cadastro único" in texto
        criterios["condicionalidades"] = []
        if "frequência escolar" in texto:
            criterios["condicionalidades"].append("frequência escolar")
        if "vacinação" in texto:
            criterios["condicionalidades"].append("vacinação")
        if "pré-natal" in texto:
            criterios["condicionalidades"].append("pré-natal")

    elif programa == "BPC":
        if "65" in texto:
            criterios["idade_min_idoso"] = 65
        if "1/4" in texto or "um quarto" in texto:
            criterios["renda_per_capita_max_frac_salario_minimo"] = 0.25
        criterios["cad_unico_required"] = "cadastro único" in texto
        if "aposentadoria" in texto or "pensão" in texto:
            criterios["no_acumulo"] = ["aposentadoria", "pensão"]

    elif programa == "Pe-de-Meia":
        if re.search(r"14\s*a\s*24", texto):
            criterios["faixa_etaria"] = "14 a 24 anos"
        elif "19" in texto and "eja" in texto:
            criterios["faixa_etaria_eja"] = "19 a 24 anos"

        criterios["cad_unico_required"] = "cadastro único" in texto
        if "meio salário mínimo" in texto:
            criterios["renda_per_capita_max_frac_salario_minimo"] = 0.5

        # Matrícula em escola pública
        criterios["matricula_publica_required"] = "escola pública" in texto or "ensino médio" in texto

        # Frequência mínima
        if "80%" in texto or "oitenta por cento" in texto:
            criterios["frequencia_minima_percent"] = 80

        # CPF obrigatório
        criterios["cpf_required"] = "cpf" in texto

        # Sem necessidade de inscrição manual
        criterios["inscricao_automatica"] = "automática" in texto or "sem inscrição" in texto
    return criterios

### Montando JSON de critérios

In [22]:
todos_criterios = {}
for programa, url in urls.items():
    texto = extrair_texto(url)
    criterios = extrair_criterios(programa, texto)
    todos_criterios[programa] = criterios

### Salvando em arquivo JSON

In [23]:
with open("criterios.json", "w", encoding="utf-8") as f:
    json.dump(todos_criterios, f, indent=2, ensure_ascii=False)

print("Critérios extraídos e salvos em criterios.json")

Critérios extraídos e salvos em criterios.json


In [24]:
# Carregar os critérios salvos
with open("criterios.json", "r", encoding="utf-8") as f:
    criterios_programas = json.load(f)

# Dados de exemplo
dados = {
    "Nome": ["Maria Silva", "João Santos", "Ana Costa"],
    "Idade": [32, 67, 25],
    "Sexo": ["F", "M", "F"],
    "Filhos": [2, 0, 1],
    "Renda Familiar": [1200, 800, 2000],
    "Pessoas no domicílio": [5, 2, 3],
    "Responsável": [True, False, True]
}

df = pd.DataFrame(dados)

# Função para calcular renda per capita
def calcular_renda_per_capita(renda, pessoas):
    return renda / pessoas

# Função para classificar cada pessoa
def classificar_categoria(linha):
    categorias = []
    programas = []

    renda_pc = calcular_renda_per_capita(linha["Renda Familiar"], linha["Pessoas no domicílio"])

    # Bolsa Família
    bf = criterios_programas.get("Bolsa Família", {})
    if bf.get("renda_per_capita_max") and renda_pc <= bf["renda_per_capita_max"]:
        categorias.append("Baixa Renda")
        programas.append("Bolsa Família")

    # BPC
    bpc = criterios_programas.get("BPC", {})
    if bpc.get("idade_min_idoso") and linha["Idade"] >= bpc["idade_min_idoso"]:
        if bpc.get("renda_per_capita_max_frac_salario_minimo"):
            salario_minimo = 1412  # valor 2025
            if renda_pc <= (salario_minimo * bpc["renda_per_capita_max_frac_salario_minimo"]):
                categorias.append("Terceira Idade")
                programas.append("BPC - Idoso")

    # Mãe Solo
    if (linha["Sexo"] == "F") and (linha["Filhos"] > 0) and (linha["Responsável"]):
        categorias.append("Mãe Solo")
        programas.append("Auxílio Mãe Solo")

    return pd.Series({
        "Categoria": ", ".join(categorias) if categorias else "Nenhuma",
        "Programa Social": ", ".join(programas) if programas else "Nenhum"
    })

# Aplicar classificação
df = pd.concat([df, df.apply(classificar_categoria, axis=1)], axis=1)

print("\nTabela final com classificação:")
print(df)


Tabela final com classificação:
          Nome  Idade Sexo  Filhos  Renda Familiar  Pessoas no domicílio  \
0  Maria Silva     32    F       2            1200                     5   
1  João Santos     67    M       0             800                     2   
2    Ana Costa     25    F       1            2000                     3   

   Responsável Categoria   Programa Social  
0         True  Mãe Solo  Auxílio Mãe Solo  
1        False   Nenhuma            Nenhum  
2         True  Mãe Solo  Auxílio Mãe Solo  
